# DSP-YOLO Pipeline Demo Notebook
This notebook demonstrates the full pipeline:
1. Load raw image
2. Apply DSP (denoise + CLAHE + sharpen)
3. Run YOLO on raw and enhanced images
4. Visualize outputs
5. Compare confidence scores

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from matplotlib import pyplot as plt
import json
import os
import sys
sys.path.append('../src')
from preprocess import enhance
from detector import load_model, run_detect

## Load Model & Image

In [ ]:
IMAGE_PATH = '../examples/forestfireschernobyl.png'
MODEL_PATH = '../models/best.pt'

raw = cv2.imread(IMAGE_PATH)
raw = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)

model = load_model(MODEL_PATH)
print('Loaded image and model.')

## Apply DSP Enhancement

In [ ]:
enh = enhance(cv2.cvtColor(raw, cv2.COLOR_RGB2BGR))
enh = cv2.cvtColor(enh, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1); plt.imshow(raw); plt.title('Raw Image'); plt.axis('off')
plt.subplot(1,2,2); plt.imshow(enh); plt.title('DSP Enhanced'); plt.axis('off')

## Run YOLO Detection

In [ ]:
raw_out, raw_vis = run_detect(model, raw)
enh_out, enh_vis = run_detect(model, enh)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1); plt.imshow(raw_vis); plt.title('Raw YOLO Output'); plt.axis('off')
plt.subplot(1,2,2); plt.imshow(enh_vis); plt.title('Enhanced YOLO Output'); plt.axis('off')

## Compare Confidence Scores

In [ ]:
def summarize(dets):
    if not dets: return {'n': 0, 'mean_conf': 0.0}
    confs = [d['score'] for d in dets]
    return {'n': len(dets), 'mean_conf': float(np.mean(confs))}

print('RAW :', summarize(raw_out))
print('ENH :', summarize(enh_out))